**EXP**-**1-  Build CNN model for sample dataset**

In [ ]:
!pip install opendatasets -qq
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import opendatasets as od
od.download('https://www.kaggle.com/datasets/hemendrasr/pizza-vs-ice-cream')
train_dir = r'/content/pizza-vs-ice-cream/dataset/train'
test_dir = r'/content/pizza-vs-ice-cream/dataset/test'
val_dir = r'/content/pizza-vs-ice-cream/dataset/valid'
train_generator = image_dataset_from_directory(train_dir, image_size=(128, 128), batch_size=32)
test_generator = image_dataset_from_directory(test_dir, image_size=(128, 128), batch_size=32)
val_generator = image_dataset_from_directory(val_dir, image_size=(128, 128), batch_size=32)
plt.figure(figsize=(10, 10))
for images, labels in val_generator:
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")
model = Sequential([
    Conv2D(64, (3, 3), input_shape=(128, 128, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(512, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()
plt.title('Training Log')
plt.plot(logs.history['loss'], label='Training Loss')
plt.plot(logs.history['accuracy'], label='Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Score')
plt.legend()
plt.show()
plt.title('Validation Log')
plt.plot(logs.history['val_loss'], label='Validation Loss')
plt.plot(logs.history['val_accuracy'], label='Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Score')
plt.legend()
plt.show()
model.evaluate(test_generator)
y_pred = model.predict(test_generator)
y_pred_classes = np.where(y_pred > 0.5, 1, 0)
y_true = np.concatenate([y for x, y in test_generator], axis=0)
import numpy as np
y_pred_classes = np.where(y_pred > 0.5, 1, 0)
y_true = np.concatenate([y for x, y in test_generator], axis=0)
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
print('Classification Report')
print(classification_report(y_true, y_pred_classes))

**EXP-2-Implement transfer learning with pretrained CNN model**

In [ ]:
!pip install -qq opendatasets

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import *
from keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import opendatasets as od
from tensorflow.keras import layers, models
import tensorflow_hub as hub

od.download("https://www.kaggle.com/datasets/hemendrasr/pizza-vs-ice-cream")

train_dir = '/content/pizza-vs-ice-cream/dataset/train'
test_dir = '/content/pizza-vs-ice-cream/dataset/test'

from keras.utils import image_dataset_from_directory
train_dataset = image_dataset_from_directory(train_dir, image_size=(600, 600), batch_size=32)
test_dataset = image_dataset_from_directory(test_dir, image_size=(600, 600), batch_size=32)

plt.figure(figsize=(10, 10))
for images, labels in test_dataset:
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

input_shape = (600, 600, 3)

base_model = tf.keras.applications.VGG16(
    input_shape=input_shape,
    include_top=False,
    weights='imagenet'
)

base_model = tf.keras.applications.InceptionV3(
    input_shape=input_shape,
    include_top=False,
    weights='imagenet'
)

base_model = tf.keras.applications.ResNet50(
    input_shape=input_shape,
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

resnet = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

resnet.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = resnet.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=1
)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

resnet.evaluate(test_dataset)

y_pred = resnet.predict(test_dataset)
y_pred_classes = np.where(y_pred > 0.5, 1, 0)
# y_pred_classes = np.argmax(y_pred, axis=1)  # For multi-class classification
y_true = np.concatenate([y for x, y in test_dataset], axis=0)

cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

print('Classification Report')
print(classification_report(y_true, y_pred_classes))

**EXP-3-Implement transfer learning for imge classification with CIFAR-10 dataset**

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
output = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=1,
                    validation_data=(test_images, test_labels))

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.concatenate(test_labels, axis=0)

cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
# Classification Report
print('Classification Report')
print(classification_report(y_true, y_pred_classes))

**EXP-4-Apply transfer learning for dog breed identification dataset**

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hartman/dog-breed-identification")

print("Path to dataset files:", path)

import pandas as pd
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data=pd.read_csv(r'C:\Users\srujo\.cache\kagglehub\datasets\hartman\dog-breed-identification\versions\1\labels.csv')
data.head()

data['id']=data['id'].apply(lambda x:x+'.jpg')
data.head()

size=(224,224)

datagen=ImageDataGenerator(rescale=1./255,
                           rotation_range=20,
                           shear_range=0.2,
                           zoom_range=0.2,
                           validation_split=0.3,
                           fill_mode='nearest',
                           horizontal_flip=True)

train=datagen.flow_from_dataframe(data,directory=r'C:\Users\srujo\.cache\kagglehub\datasets\hartman\dog-breed-identification\versions\1\train',
                                  x_col='id',
                                  y_col='breed',
                                  class_mode='categorical',
                                  target_size=size,
                                  batch_size=32,
                                  subset='training')

test=datagen.flow_from_dataframe(data,directory=r'C:\Users\srujo\.cache\kagglehub\datasets\hartman\dog-breed-identification\versions\1\train',
                                 x_col='id',
                                 y_col='breed',
                                 class_mode='categorical',
                                 target_size=size,
                                 batch_size=32,
                                 subset='validation')

import tensorflow

vgg=VGG16(weights='imagenet',include_top=False,input_shape=(64,64,3))
vgg.trainable=False
model=vgg.output
model=GlobalAveragePooling2D()(model)
model=Dropout(0.2)(model)
model=Dense(512,activation='relu')(model)
pred=Dense(120,activation='softmax')(model)
mm=tensorflow.keras.models.Model(vgg.input,pred)

mm.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
er=EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)
his=mm.fit(train,epochs=5,validation_data=test,callbacks=[er])

import matplotlib.pyplot as plt
img, label = next(train)

fig = plt.figure(figsize=(15, 10))

for i in range(12):
    fig.add_subplot(3, 4, i+1)
    plt.imshow(img[i])
    plt.axis('off')

**EXP-5-Build review sentiment classifier using transfer learning**

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import collections
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim.models import Word2Vec

import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tabulate import tabulate

import pandas as pd

df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')

df_train.head()

df_train.info()

print(df_train['1'].value_counts())
print(df_test['1'].value_counts())

# Preprocess text
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    text = [word for word in text.split() if word not in stop_words]
    return ' '.join(text)

df_train['processed_text'] = df_train['0'].apply(preprocess_text)
df_test['processed_text'] = df_test['0'].apply(preprocess_text)

# Tokenize and pad sequences
vocab_size = 10000
max_len = 200  # maximum length of a sequence
embedding_dim = 16

tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(df_train['processed_text'])

train_seq = tokenizer.texts_to_sequences(df_train['processed_text'])
test_seq = tokenizer.texts_to_sequences(df_test['processed_text'])

train_pad = pad_sequences(train_seq, maxlen=max_len, padding='post')
test_pad = pad_sequences(test_seq, maxlen=max_len, padding='post')

from tensorflow.keras.utils import to_categorical

train_label = to_categorical(df_train['1'])
test_label = to_categorical(df_test['1'])

y_train = df_train['1'].values
y_train

# Build the model
model = tf.keras.models.Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_pad, df_train['1'].values, epochs=10, batch_size=64, validation_split=0.1)

# Plot training history

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()


plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='lower right')
plt.show()

# Evaluate the model
test_seq_padded = pad_sequences(test_seq, maxlen=max_len, padding='post')
test_seq_np = np.array(test_seq_padded)

predictions = model.predict(test_seq_np)
predicted_labels = np.round(predictions)
true_labels = np.array(df_test['1'])

cm = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", annot_kws={"size": 16})
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

table = [
    ["Accuracy", accuracy],
    ["Precision", precision],
    ["Recall", recall],
    ["F1-score", f1]
]

print(tabulate(table, headers=["Metric", "Value"], tablefmt="fancy_grid"))

**EXP-6-Apply transfer learning for IMDB dataset with word embeddings**

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import collections
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim.models import Word2Vec

import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tabulate import tabulate

import pandas as pd

df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')

df_train.head()

df_train.info()

print(df_train['1'].value_counts())
print(df_test['1'].value_counts())

# Preprocess text
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    text = [word for word in text.split() if word not in stop_words]
    return ' '.join(text)

df_train['processed_text'] = df_train['0'].apply(preprocess_text)
df_test['processed_text'] = df_test['0'].apply(preprocess_text)

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=df_train['processed_text'], vector_size=100, window=5, min_count=1, workers=4)
word2vec_model.save("word2vec.model")

# Create embedding matrix
vocab_size = 10000
embedding_dim = 100
word_index = {word: i+1 for i, word in enumerate(word2vec_model.wv.index_to_key)}

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if i < vocab_size:
        try:
            embedding_vector = word2vec_model.wv[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), embedding_dim)

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(df_train['processed_text'])

train_seq = tokenizer.texts_to_sequences(df_train['processed_text'])
test_seq = tokenizer.texts_to_sequences(df_test['processed_text'])

average_len = np.mean([len(seq) for seq in train_seq])
max_len = int(average_len + 100)

train_pad = pad_sequences(train_seq, maxlen=max_len, padding='post')
test_pad = pad_sequences(test_seq, maxlen=max_len, padding='post')

from tensorflow.keras.utils import to_categorical

train_label = to_categorical(df_train['1'])
test_label = to_categorical(df_test['1'])

y_train = df_train['1'].values
y_train

# Build the model
model = tf.keras.models.Sequential([
    Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_pad, df_train['1'].values, epochs=10, batch_size=64, validation_split=0.1)

# Plot training history
plt.style.use('dark_background')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

plt.style.use('dark_background')
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='lower right')
plt.show()



# Evaluate the model
test_seq_padded = pad_sequences(test_seq, maxlen=max_len, padding='post')
test_seq_np = np.array(test_seq_padded)

predictions = model.predict(test_seq_np)
predicted_labels = np.round(predictions)
true_labels = np.array(df_test['1'])

cm = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", annot_kws={"size": 16})
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

table = [
    ["Accuracy", accuracy],
    ["Precision", precision],
    ["Recall", recall],
    ["F1-score", f1]
]

print(tabulate(table, headers=["Metric", "Value"], tablefmt="fancy_grid"))

**EXP-7-Create document summaries using transfer learning**

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
train_data, test_data = imdb['train'], imdb['test']

train_sentences = []
train_labels = []

for s, l in train_data:
    train_sentences.append(str(s.numpy()))
    train_labels.append(l.numpy())

test_sentences = []
test_labels = []

for s, l in test_data:
    test_sentences.append(str(s.numpy()))
    test_labels.append(l.numpy())

vocab_size = 10000
max_length = 200
embedding_dim = 16
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
num_epochs = 5
model.fit(train_padded, train_labels, epochs=num_epochs, validation_data=(test_padded, test_labels))

# Save the trained model
model.save("/content/drive/MyDrive/Temp/imdb_model.h5")

"""# Use Trained Model To implement TL"""

import tensorflow.keras.backend as K
from nltk.tokenize import sent_tokenize
import tensorflow as tf
import nltk
nltk.download('punkt')

imdb_model = tf.keras.models.load_model('/content/drive/MyDrive/Temp/imdb_model.h5')

for layer in imdb_model.layers:
    layer.trainable = False

corpus = [
    "The movie had a very strong start.",
    "However, the plot quickly fell apart.",
    "The acting was top-notch, especially the lead actor.",
    "But the storyline was predictable and uninspiring.",
    "Overall, the movie had good moments but was disappointing."
]

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(corpus)

sequences = tokenizer.texts_to_sequences(corpus)
max_length = 200
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

max_length = 200
input_shape = (max_length,)
inputs = tf.keras.Input(shape=input_shape)

x = imdb_model(inputs)

x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

summarization_model = tf.keras.Model(inputs=inputs, outputs=x)

summarization_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

summarization_model.summary()

predictions = summarization_model.predict(padded_sequences)

summary = [corpus[i] for i, score in enumerate(predictions) if score > 0.436]

print("Summary:")
for sentence in summary:
    print(sentence)

**EXP-8-Perform Audio event classification with transfer learning**

In [ ]:
import pandas as pd

df= pd.read_csv('/content/urbansound8k/data.csv')
df.head(5)

import librosa
audio_file_path='/content/urbansound8k/fold10/100648-1-0-0.wav'
librosa_audio_data,librosa_sample_rate=librosa.load(audio_file_path)

print(librosa_audio_data)

import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5))
plt.plot(librosa_audio_data)

# @title Import Modules
import pandas as pd
import numpy as np
import os, librosa
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import IPython.display as ipd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from keras.utils import to_categorical
from keras import layers, Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from sklearn.metrics import confusion_matrix
import numpy as np
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

mfccs = librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc=40)

mfccs

# @title DataPath
audio_dataset_path = '/content/data'
metadata =  pd.read_csv('/content/urbansound8k/data.csv')
metadata.head()

def mfccExtract(file):
    waveform, sampleRate = librosa.load(file_name)
    features = librosa.feature.mfcc(y = waveform, sr = sampleRate, n_mfcc = 50)
    return np.mean(features, axis = 1)

extractAll = []

for index_num, row in tqdm(metadata.iterrows()):
    # Constructing file path
    file_name = os.path.join(audio_dataset_path, 'fold' + str(row['fold']), row['slice_file_name'])

    # Extracting features and appending them
    features = mfccExtract(file_name)
    extractAll.append([features, row['class']])

featuresDf = pd.DataFrame(extractAll, columns = ['Features', 'Class'])
featuresDf.head()

X=np.array(featuresDf['Features'].tolist())
Y=np.array(featuresDf['Class'].tolist())

labelencoder=LabelEncoder()
Y=to_categorical(labelencoder.fit_transform(Y))

Y

### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)
num_labels=Y.shape[1]

from tensorflow.keras.applications import VGG19
from keras.layers import *
base_model = VGG19(weights='imagenet',
                       include_top=False,
                       input_shape=(32, 32, 3))

from tensorflow.keras.models import  Model
# Add classification layers on top of it
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu',input_shape = (50,))(x)
output = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

history = model.fit(X_train, Y_train, validation_data = (X_test, Y_test), epochs = 10)

test_accuracy=model.evaluate(X_test,Y_test,verbose=0)
print(test_accuracy[1])

historyDf = pd.DataFrame(history.history)

historyDf.loc[:, ['loss', 'val_loss']].plot()

historyDf.loc[:, ['accuracy', 'val_accuracy']].plot()

# Evaluating model
score = model.evaluate(X_test, Y_test)[1] * 100
print(f'Validation accuracy of model : {score:.2f}%')